<a href="https://colab.research.google.com/github/JTT94/nlp_neural_process/blob/master/Class_BERT_NeuralProcesses_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re

from keras import backend as K
import numpy as np
import string
from datetime import datetime 
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

W0526 19:24:22.406549 140326760654720 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization


     |████████████████████████████████| 71kB 8.8MB/s 


In [0]:
# initialiase tensorboard 
# from https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab


# Get TensorBoard running in the background. 
LOG_DIR = './test_output'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [4]:
# # #Download and unzip ngrok. 
!test -e ngrok-stable-linux-amd64.zip || wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!test -e ngrok || unzip ngrok-stable-linux-amd64.zip


--2019-05-26 19:24:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.173.3.255, 52.55.191.55, 52.203.66.95, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.173.3.255|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  15.6MB/s    in 1.0s    

2019-05-26 19:24:31 (15.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
# #Launch ngrok background process...
get_ipython().system_raw('./ngrok http 6006 &')

In [6]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://5d1da96c.ngrok.io


In [7]:
import os, sys
sys.path.append('../') # add personal code dir to path for import


!test -d neural_process || git clone https://github.com/JTT94/nlp_neural_process.git neural_process
sys.path.append('./neural_process/')

import random
from neural_process import split_context_target, NeuralProcessParams
from neural_process.network import *
from neural_process.loss import *
from neural_process.predict import *
from neural_process.process import *

from neural_process.tf_model_builder_AUC import *
from neural_process.bert_utils import *

Cloning into 'neural_process'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 120 (delta 73), reused 101 (delta 61), pack-reused 0
Receiving objects: 100% (120/120), 680.93 KiB | 1.40 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [8]:
# Initialize session
sess = tf.Session()
K.set_session(sess)
K.tensorflow_backend._get_available_gpus()


['/job:localhost/replica:0/task:0/device:GPU:0']

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# filename = './cleaned_data.csv'
# filename = './data1.csv'
# df = pd.read_csv(filename, index_col=0)
# cols = ['comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# cols = ['comment','cleaned_comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# df = df[cols]

#score_column = ['antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# text_col_name = 'cleaned_comment'
# text_col_name = 'comment'

#--------

### For kaggle dataset

## Training data

# filename = './gdrive/My Drive/Data/train.csv'
train_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_train.csv'

df = pd.read_csv(train_filename)
cols = ['comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df = df[cols]

score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

text_col_name = 'comment_text'

#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df[i] = pd.to_numeric(df[i],downcast='float')

# over represent toxic comments

df['num_toxic_atts'] = df[cols[1:]].apply(lambda x: np.sum(x), axis = 1)

df_toxic = df[df.num_toxic_atts  > 0]
# print(len(df_toxic))

df_healthy_sample = df[df.num_toxic_atts == 0].sample(frac=1.0)[:len(df_toxic)]
# print(len(df_healthy_sample))

df_train = pd.concat([df_toxic, df_healthy_sample]).sample(frac=1.0)


## Test data:

test_data_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_test.csv'
test_labels_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_test_labels.csv'
test_df = pd.read_csv(test_data_filename)
test_labels = pd.read_csv(test_labels_filename)
test_df = pd.merge(test_df, test_labels, on='id')
#get rid of -1 values
test_df = test_df.replace(-1, np.nan).dropna(subset=cols)

#restrict comment length
test_df = test_df[test_df.comment_text.str.len() <= 250]

test_df = test_df[cols]
#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  test_df[i] = pd.to_numeric(test_df[i],downcast='float')

In [11]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)

Instructions for updating:
Colocations handled automatically by placer.


W0526 19:25:27.243856 140326760654720 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0526 19:25:29.443532 140326760654720 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
# #Pre process data for bert embedding
max_seq_length = 128

# import pickle

# # train_input_examples = create_examples(df_train, score_column, text_col_name)
# # test_input_examples = create_examples(df_test, score_column, text_col_name)

# # train_features = convert_examples_to_features(train_input_examples, max_seq_length, tokenizer)
# # test_features = convert_examples_to_features(test_input_examples, max_seq_length, tokenizer)

# # pickle.dump(train_features, open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'wb'))
# # pickle.dump(test_features, open('./gdrive/My Drive/Kaggle_toxic_comments/test_features.p', 'wb'))

# # ## Load features previously saved
# train_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))
# test_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))

In [0]:
# utility methods
from collections import namedtuple
NeuralProcessParams = namedtuple('NeuralProcessParams', ['dim_z', 'n_hidden_units_h', 'n_hidden_units_g'])
GaussianParams = namedtuple('GaussianParams', ['mu', 'sigma'])


def batch_mlp(input, inner_layer_dims, output_dim, variable_scope):
  """Apply MLP to the final axis of a 3D tensor (reusing already defined MLPs).
  
  Args:
    input: input tensor of shape [B,n,d_in].
    output_sizes: An iterable containing the output sizes of the MLP as defined 
        in `basic.Linear`.
    variable_scope: String giving the name of the variable scope. If this is set
        to be the same as a previously defined MLP, then the weights are reused.
    
  Returns:
    tensor of shape [B,n,d_out] where d_out=output_sizes[-1]
  """
  # Get the shapes of the input and reshape to parallelise across observations

  output = input

  
  # Pass through MLP
  with tf.variable_scope(variable_scope, reuse=tf.AUTO_REUSE):
    for i, size in enumerate(inner_layer_dims):
      output = tf.nn.relu(
          tf.layers.dense(output, size, name="layer_{}".format(i)))

    # Last layer without a ReLu
    output = tf.layers.dense(output, output_dim, name="layer_{}".format(i + 1))

  return output

In [0]:
class Embedder(object):
  
  def __init__(self, BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", trainable=True):
    self.BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
    self.tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)
    self.trainable = trainable
    
  def __call__(self, input_ids, input_mask, segment_ids):
    embedder = hub.Module(self.BERT_model_hub,trainable=self.trainable)
    bert_inputs = dict(input_ids=input_ids,
                       input_mask=input_mask, 
                       segment_ids=segment_ids)

    bert_outputs = embedder(inputs=bert_inputs,
                               signature="tokens", 
                               as_dict=True)
    
  # Use "pooled_output" for classification tasks on an entire sentence. Use "sequence_outputs" for token-level output
    return bert_outputs["pooled_output"]
  

In [0]:
class Decoder(object):
  """The Decoder."""

  def __init__(self, layer_dims, num_classes):
    self.layer_dims = layer_dims
    self.num_classes = num_classes
    
  def __call__(self, input_xs_embedding, z_samples):
  
        # inputs dimensions
    # z_sample has dim [n_draws, dim_z]
    # x_star has dim [N_star, dim_x]
    n_draws = z_samples.get_shape().as_list()[0]
    n_xs = tf.shape(input_xs_embedding)[0]

    # Repeat z samples for each x*
    #z_samples_repeat = tf.expand_dims(z_samples, axis=1)

    #z_samples_repeat = tf.expand_dims(z_samples, axis=1)
    z_samples_repeat = tf.tile(z_samples, [1, n_xs, 1])

    # Repeat x* for each z sample
    x_star_repeat = tf.expand_dims(input_xs_embedding, axis=0)
    x_star_repeat = tf.tile(x_star_repeat, [n_draws, 1, 1])

    # Concatenate x* and z
    inputs = tf.concat([x_star_repeat, z_samples_repeat], axis=2)

    # decoder mlp
    inner_layer_dims = self.layer_dims
    output_dim = self.num_classes *2
    hidden = batch_mlp(inputs, inner_layer_dims, output_dim, "decoder")

    # Get the mean an the variance
    mu, log_sigma = tf.split(hidden, 2, axis= -1)

    # Bound the variance
    sigma_star = 0.1 + 0.9 * tf.nn.softplus(log_sigma)
    mu_star = tf.math.sigmoid(mu)


    return GaussianParams(mu_star, sigma_star)




In [0]:
class Encoder(object):

  def __init__(self, layer_dims, latent_dim):
    self.layer_dims = layer_dims
    self.latent_dim = latent_dim
    
  def __call__(self, xs, ys):
    print(xs)
    print(ys)
    xys = tf.concat([xs, ys], axis=1)


    # encoder mlp
    inner_layer_dims = self.layer_dims[:-1]
    output_dim = self.layer_dims[-1]
    rs = batch_mlp(xys, inner_layer_dims, output_dim, "encoder")
    
    # aggregate rs
    r = self._aggregate_r(rs)
    
    # get mu and sigma
    z_params = self._get_z_params(r)
    
    # distribution
    dist = tfd.MultivariateNormalDiag(loc=z_params.mu,
                                          scale_diag=z_params.sigma)
    return dist
    
  def _aggregate_r(self, context_rs: tf.Tensor) -> tf.Tensor:
    """Aggregate the output of the encoder to a single representation

    Creates an aggregation (mean) operator to combine the encodings of multiple context inputs

    Parameters
    ----------
    context_rs
        Input encodings tensor, shape: (n_samples, dim_r)

    Returns
    -------
        Output tensor of aggregation result
    """
    mean = tf.reduce_mean(context_rs, axis=0)
    r = tf.reshape(mean, [1, -1])
    return r
  
  def _get_z_params(self, context_r: tf.Tensor) -> GaussianParams:
    """Map encoding to mean and covariance of the random variable Z

    Creates a linear dense layer to map encoding to mu_z, and another linear mapping + a softplus activation for Sigma_z

    Parameters
    ----------
    context_r
        Input encoding tensor, shape: (1, dim_r)
    params
        Neural process parameters

    Returns
    -------
        Output tensors of the mappings for mu_z and Sigma_z
    """
    hidden = context_r
    with tf.variable_scope("latent_encoder", reuse=tf.AUTO_REUSE):
      # First apply intermediate relu layer 
      hidden = tf.nn.relu(
          tf.layers.dense(hidden, 
                          (self.layer_dims[-1] + self.latent_dim)/2, 
                          name="penultimate_layer"))
      
      # Then apply further linear layers to output latent mu and log sigma
      mu = tf.layers.dense(hidden, self.latent_dim, name="mean_layer")
      log_sigma = tf.layers.dense(hidden, self.latent_dim, name="std_layer")
      

    # Compute sigma
    sigma = 0.1 + 0.9 * tf.sigmoid(log_sigma)

    return GaussianParams(mu, sigma)

In [0]:
class NLP_NeuralProcess(object):
  
  def __init__(self,
                  score_col, 
                  params = NeuralProcessParams(dim_z=20, 
                                                     n_hidden_units_h=[128, 128, 128], 
                                                     n_hidden_units_g=[128, 128, 128]),
                  num_classes = 6, 
                  num_draws = 2, 
                  lr = 2e-5,
                  batch_size = 32, 
                  num_warmup_steps=100,
                  num_train_steps = 10**3,
                  save_summary_steps = 100,
                  save_checkpoints_steps = 500,
                  output_dir = "./test_output"
                 ):
    
    self.params = params
    self.encoder = Encoder(layer_dims = self.params.n_hidden_units_h, 
                           latent_dim=self.params.dim_z)
    self.decoder = Decoder(layer_dims= self.params.n_hidden_units_g, 
                           num_classes=num_classes)
    self.num_draws = num_draws
#     self.estimator = None
    #self.embedder = Embedder()
    #####  
    num_labels = len(score_col)
    
    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(model_dir=output_dir,
        save_summary_steps=save_summary_steps, save_checkpoints_steps=save_checkpoints_steps)

    #####


    model_fn = self.model_fn_builder(num_labels = num_labels, learning_rate=lr,
      num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)

    self.estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,
      params={"batch_size": batch_size})
    
  def create_model(self, 
                   target_input_ids, 
                   target_input_mask, 
                   target_segment_ids, 
                   target_scores=None,
                   context_input_ids = None, 
                   context_input_mask = None, 
                   context_segment_ids= None, 
                   context_scores = None
                   ):
    
    # apply embedder
    embedder = hub.Module(BERT_model_hub,trainable=False)
    
    valid_context = (context_input_ids is not None) & (context_input_mask is not None) & (context_segment_ids is not None) & (context_scores is not None)
    
    # target processing - all scenarios
    target_inputs = dict(input_ids=target_input_ids,
                     input_mask=target_input_mask, 
                     segment_ids=target_segment_ids)
    target_embeddings = embedder(inputs=target_inputs,
                               signature="tokens", 
                               as_dict=True)
    target_xs = target_embeddings["pooled_output"]
    
    
    if valid_context:
      
      # context processing - training
      context_inputs = dict(input_ids=context_input_ids,
                         input_mask=context_input_mask, 
                         segment_ids=context_segment_ids)
      context_embeddings = embedder(inputs=context_inputs,
                                 signature="tokens", 
                                 as_dict=True)
      context_xs = context_embeddings["pooled_output"]
      context_ys = context_scores
      # total x,y 
      x_all = tf.concat([context_xs, target_xs], axis=0)
      
      # get encoding params with context
      context_z_dist = self.encoder(context_xs, context_ys)
      # predictions with context
      posterior_pred = self.decoder(target_xs, context_z_dist.sample(self.num_draws))
        
       # target scores - context training / evaluation
      if target_scores is not None:
        target_ys = target_scores
        y_all = tf.concat([context_ys, target_ys], axis=0)
        all_z_dist = self.encoder(x_all, y_all)
        
        # loss
        loglike = self.loglikelihood(target_ys, posterior_pred)
        KL_loss = self.KLqp_gaussian(all_z_dist.parameters['loc'], 
                                     all_z_dist.parameters['scale_diag'], 
                                     context_z_dist.parameters['loc'], 
                                     context_z_dist.parameters['scale_diag'])
        loss = tf.negative(loglike) + KL_loss
        # context and training / evaluation
        return (loss, posterior_pred, target_ys)
      
      # context prediction
      return  (None, posterior_pred, None)
    
    # no context
    else:
      x_all = target_xs 
      # get internal representation
      mean_zero = tf.constant(np.repeat(0., params.dim_z))
      epsilon_dist = tfd.MultivariateNormalDiag(loc= mean_zero)                            
      epsilon = tf.expand_dims(epsilon_dist.sample(self.num_draws), axis=1)
      epsilon = tf.cast(epsilon, tf.float32)
      prior_predict = self.decoder(x_all, epsilon)
      
      # target scores - no context training / evaluation
      if target_scores is not None:
        target_ys = target_scores
        loglike = self.loglikelihood(target_ys, prior_predict)
        loss = tf.negative(loglike)
        # no context/ training / evaluation
        return  (loss, prior_predict, target_ys)
   
    
      # no context prediction
      return  (None, prior_predict, None)

  
  def loglikelihood(self, y_star: tf.Tensor, dist):
    """Log-likelihood of an output given a predicted """
    p_normal = tfd.MultivariateNormalDiag(loc = dist.mu, scale_diag=dist.sigma)
    loglike = p_normal.log_prob(y_star)
    loglike = tf.reduce_sum(loglike, axis=0)
    loglike = tf.reduce_mean(loglike)
    return loglike
  
  def KLqp_gaussian(self, mu_q: tf.Tensor, sigma_q: tf.Tensor, mu_p: tf.Tensor, sigma_p: tf.Tensor) -> tf.Tensor:
    """Kullback-Leibler divergence between two Gaussian distributions

    Determines KL(q || p) = < log( q / p ) >_q

    Parameters
    ----------
    mu_q
        Mean tensor of distribution q, shape: (1, dim)
    sigma_q
        Variance tensor of distribution q, shape: (1, dim)
    mu_p
        Mean tensor of distribution p, shape: (1, dim)
    sigma_p
        Variance tensor of distribution p, shape: (1, dim)

    Returns
    -------
        KL tensor, shape: (1)
    """
    sigma2_q = tf.square(sigma_q) + 1e-16
    sigma2_p = tf.square(sigma_p) + 1e-16
    temp = sigma2_q / sigma2_p + tf.square(mu_q - mu_p) / sigma2_p - 1.0 + tf.log(sigma2_p / sigma2_q + 1e-16)
    return 0.5 * tf.reduce_sum(temp)
  
  def context_target_split(self, batch_size =32):
    btch_sz = batch_size
    n_context = tf.random_shuffle(tf.range(1,btch_sz))[0]
    
    indices = tf.range(0, btch_sz)
    context_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context))
    target_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context, btch_sz))
    
    return context_set_indices, target_set_indices
    
  def model_fn_builder(self, num_labels, learning_rate, num_train_steps, num_warmup_steps):
      """Returns `model_fn` closure for TPUEstimator."""
      
      
      def model_fn(features, mode, params):  # pylint: disable=unused-argument
          """The `model_fn` for TPUEstimator."""
          
          # run model
          # -------------------------------------------------------------------------------------------
          target_input_ids = None
          target_input_mask = None
          target_segment_ids = None 
          target_scores = None
          
          context_input_ids = None 
          context_input_mask = None 
          context_segment_ids = None
          context_scores = None
          
          # training 
          if mode == tf.estimator.ModeKeys.TRAIN:
            input_ids = std_features["input_ids"]
            input_mask = std_features["input_mask"]
            segment_ids = std_features["segment_ids"]    
            scores = std_features["scores"]
            
            # context split
            context_set_indices, target_set_indices= self.context_target_split(batch_size =32)
            
            target_input_ids = tf.gather(input_ids,target_set_indices)
            target_input_mask = tf.gather(input_mask,target_set_indices)
            target_segment_ids = tf.gather(segment_ids,target_set_indices) 
            target_scores = tf.gather(scores,target_set_indices)

            context_input_ids = tf.gather(input_ids,context_set_indices) 
            context_input_mask = tf.gather(input_mask,context_set_indices) 
            context_segment_ids = tf.gather(segment_ids,context_set_indices)
            context_scores = tf.gather(scores,context_set_indices)
            
            
          elif mode == tf.estimator.ModeKeys.PREDICT:
            print('Prediction')
            target_input_ids = features["input_ids"]
            target_input_mask = features["input_mask"]
            target_segment_ids = features["segment_ids"]    
            target_scores = features["scores"]
            
            try:
              context_input_ids = features["supplied_context_input_ids"]
              context_input_mask = features["supplied_context_input_mask"]
              context_segment_ids = features["supplied_context_segment_ids"] 
              context_scores = features["supplied_context_scores"]
               
            except:
              print("****No context supplied ****")
              context_input_ids = None
              context_input_mask = None
              context_segment_ids = None
              context_scores = None
            
          else:
            print('Evaluation')
            target_input_ids = features["input_ids"]
            target_input_mask = features["input_mask"]
            target_segment_ids = features["segment_ids"]    
            target_scores = features["scores"]

            try:
              context_input_ids = features["supplied_context_input_ids"]
              context_input_mask = features["supplied_context_input_mask"]
              context_segment_ids = features["supplied_context_segment_ids"] 
              context_scores = features["supplied_context_scores"]
               
            except:
              print("****No context supplied ****")
              context_input_ids = None
              context_input_mask = None
              context_segment_ids = None
              context_scores = None


          (loss, prediction, true_y) = self.create_model(target_input_ids, 
                                                         target_input_mask, 
                                                         target_segment_ids, 
                                                         target_scores,
                                                         context_input_ids, 
                                                         context_input_mask, 
                                                         context_segment_ids, 
                                                         context_scores)

          train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
          ystar, _ = tf.nn.moments(prediction.mu,[0])
          variance, _ = tf.nn.moments(tf.math.square(prediction.sigma),[0])
          
          
          # output from model
          # -------------------------------------------------------------------------------------------

          # training 
          if mode == tf.estimator.ModeKeys.TRAIN:
              return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

          # prediction
          elif mode == tf.estimator.ModeKeys.PREDICT:
              return tf.estimator.EstimatorSpec(mode=mode, predictions={'prediction_mean': ystar, 'prediction_var': variance})

          # evaluation
          else:
            # Calculate evaluation metrics.
            eval_metrics = {}

            # AUC
            def metric_fn(pred_scores, real_scores, trait_num):
                auc_value = tf.metrics.auc(real_scores[:,trait_num], pred_scores[:,trait_num])
                accuracy_value = tf.metrics.accuracy(labels = tf.round(real_scores[:,trait_num]), predictions=tf.round(pred_scores[:,trait_num]))
                return {"auc"+str(trait_num): auc_value, "accuracy"+str(trait_num): accuracy_value}

            labels = true_y # need to round them if true labels are not 1 or 0
            eval_metrics_lst = [metric_fn(ystar, labels, trait_num) for trait_num in range(num_labels)]

            for d in eval_metrics_lst:
                tf.summary.scalar(list(d.keys())[0], list(d.values())[0][1]) # make available to tensorboard
                eval_metrics.update(d)
            return tf.estimator.EstimatorSpec(mode=mode,loss=loss, eval_metric_ops=eval_metrics)

      return model_fn
  

  def prepare_examples(self, df_train, score_column, text_col_name, supplied_context_df=None):
      num_labels = len(score_column)
      train_input_examples = create_examples(df_train, score_column, text_col_name)
      train_features = convert_examples_to_features(train_input_examples, max_seq_length, tokenizer)
      
      if supplied_context_df is not None:
        supplied_context_examples = create_examples(supplied_context_df, score_column, text_col_name)
        supplied_context_features = convert_examples_to_features(supplied_context_examples, max_seq_length, tokenizer)
        train_input_fn = input_fn_builder(
          features=train_features, seq_length=max_seq_length, 
          num_labels = num_labels, is_training=True, drop_remainder=False,
          supplied_context_features = supplied_context_features)
      
      else:
        train_input_fn = input_fn_builder(
          features=train_features, seq_length=max_seq_length, 
          num_labels = num_labels, is_training=True, drop_remainder=False)
        
      return train_input_fn
  
  def predict(self, 
            df_train, 
            score_col, 
            text_col,
            supplied_context_df=None
             ):
    
    test_input_fn = self.prepare_examples(df_train, score_col, text_col, supplied_context_df)
    preds = self.estimator.predict(input_fn=test_input_fn)
    
    return preds
  
  def evaluate(self, 
               eval_steps,
               df_train, 
               score_col, 
               text_col,
               supplied_context_df=None):
    num_labels = len(score_col)
    eval_input_fn = self.prepare_examples(df_train, score_col, text_col, supplied_context_df)
    
    result = self.estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
    return result
  
  def train(self,num_train_steps,
            df_train, 
            score_col, 
            text_col,
           ):

    # Create an input function for training. drop_remainder = True for using TPUs.
    train_input_fn = self.prepare_examples(df_train, score_col, text_col)

    print('Beginning Training!')
    current_time = datetime.now()
    self.estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    print("Training took time ", datetime.now() - current_time)

In [18]:

# tf.reset_default_graph()

score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] 
text_col_name = 'comment_text'

params = NeuralProcessParams(dim_z=2, n_hidden_units_h=[128, 128, 128], n_hidden_units_g=[128, 128, 128])


neural_process = NLP_NeuralProcess(score_col=score_column, params = params, num_draws = 2)

### Train:

# neural_process.train(df_train=df_train, 
#                      score_col= score_column, 
#                      text_col=text_col_name, 
#                      num_train_steps=10**3) 
# #                      num_warmup_steps=5)

INFO:tensorflow:Using config: {'_model_dir': './test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9fb7bef5c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0526 19:25:30.718868 140326760654720 estimator.py:201] Using config: {'_model_dir': './test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9fb7bef5c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
df_test = test_df[:100]

supp_context = test_df[100:200]

neural_process.evaluate(10**2,
                       df_test, 
                       score_col= score_column, 
                       text_col=text_col_name, supplied_context_df = supp_context)

INFO:tensorflow:Could not find trained model in model_dir: ./test_output, running initialization to evaluate.


I0526 19:25:30.881641 140326760654720 estimator.py:488] Could not find trained model in model_dir: ./test_output, running initialization to evaluate.


INFO:tensorflow:Calling model_fn.


I0526 19:25:31.025637 140326760654720 estimator.py:1111] Calling model_fn.


Evaluation
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Finished evaluation at 2019-05-26-19:26:37


I0526 19:26:37.535040 140326760654720 evaluation.py:277] Finished evaluation at 2019-05-26-19:26:37


INFO:tensorflow:Saving dict for global step 0: accuracy0 = 0.8596875, accuracy1 = 0.0, accuracy2 = 0.215, accuracy3 = 0.9528125, accuracy4 = 0.0903125, accuracy5 = 0.03, auc0 = 0.48214254, auc1 = 1.0, auc2 = 0.45062232, auc3 = 0.5161392, auc4 = 0.41402495, auc5 = 0.12006351, global_step = 0, loss = 10.404034


I0526 19:26:37.538020 140326760654720 estimator.py:1979] Saving dict for global step 0: accuracy0 = 0.8596875, accuracy1 = 0.0, accuracy2 = 0.215, accuracy3 = 0.9528125, accuracy4 = 0.0903125, accuracy5 = 0.03, auc0 = 0.48214254, auc1 = 1.0, auc2 = 0.45062232, auc3 = 0.5161392, auc4 = 0.41402495, auc5 = 0.12006351, global_step = 0, loss = 10.404034


{'accuracy0': 0.8596875,
 'accuracy1': 0.0,
 'accuracy2': 0.215,
 'accuracy3': 0.9528125,
 'accuracy4': 0.0903125,
 'accuracy5': 0.03,
 'auc0': 0.48214254,
 'auc1': 1.0,
 'auc2': 0.45062232,
 'auc3': 0.5161392,
 'auc4': 0.41402495,
 'auc5': 0.12006351,
 'global_step': 0,
 'loss': 10.404034}

In [0]:
preds = neural_process.predict(
                       df_test, 
                       score_col= score_column, 
                       text_col=text_col_name,supplied_context_df = supp_context)

In [21]:
i=0
for pred in preds:
  i += 1
  if i<100:
    val = df_test.reset_index().loc[[i],:]
    print(val['comment_text'])
    print(val.values)
    print(pred)
  else:
    break

INFO:tensorflow:Could not find trained model in model_dir: ./test_output, running initialization to predict.


I0526 19:26:39.618526 140326760654720 estimator.py:604] Could not find trained model in model_dir: ./test_output, running initialization to predict.


INFO:tensorflow:Calling model_fn.


I0526 19:26:39.751532 140326760654720 estimator.py:1111] Calling model_fn.


Prediction
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


I0526 19:26:47.544112 140326760654720 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0526 19:26:47.697426 140326760654720 session_manager.py:493] Done running local_init_op.


1    :Dear god this site is horrible.
Name: comment_text, dtype: object
[[7 ':Dear god this site is horrible.' 0.0 0.0 0.0 0.0 0.0 0.0]]
{'prediction_mean': array([0.47745132, 0.6166792 , 0.5257212 , 0.52102816, 0.5961162 ,
       0.50846773], dtype=float32), 'prediction_var': array([0.8630141 , 0.43919033, 0.53195393, 0.6121141 , 0.37925267,
       0.56666386], dtype=float32)}
2    this other one from 1897
Name: comment_text, dtype: object
[[16 'this other one from 1897' 0.0 0.0 0.0 0.0 0.0 0.0]]
{'prediction_mean': array([0.48166043, 0.6161827 , 0.49238142, 0.53094643, 0.5957934 ,
       0.50483537], dtype=float32), 'prediction_var': array([0.9879074 , 0.4267248 , 0.51427376, 0.65802014, 0.3515126 ,
       0.54139686], dtype=float32)}
3    == Reason for banning throwing == \n\n This ar...
Name: comment_text, dtype: object
[[17
  '== Reason for banning throwing == \n\n This article needs a section on /why/ throwing is banned. At the moment, to a non-cricket fan, it seems kind of arbit

In [0]:
! cp -r ./test_output "./gdrive/My Drive/Data/"

In [0]:
print("***** Eval results *****")
for key in sorted(result.keys()):
  print('  {} = {}'.format(key, str(result[key])))
      

In [0]:
predict_features = test_features[:100]

In [0]:
test_input_fn = input_fn_builder(
    features=predict_features, seq_length=max_seq_length, 
    num_labels = num_labels, is_training=False, drop_remainder=False)

In [0]:
# test_scores = np.array([predict_features[i].score for i, val in enumerate(predict_features)])

preds = estimator.predict(input_fn=test_input_fn)
preds_vals = list(preds)
pred_means = np.array([val['prediction_mean'] for key, val in enumerate(preds_vals)])

In [0]:
def total_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  correct_guesses = np.reshape(preds == test_scores,-1)
  return np.sum(correct_guesses) / len(correct_guesses)

In [0]:
def class_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  correct_guesses = preds == test_scores
  return correct_guesses.sum(axis=0)/ len(correct_guesses)


In [0]:
print(total_accuaracy(pred_means, test_scores))
print(class_accuaracy(pred_means, test_scores))

In [0]:
preds_vals

In [0]:
test_scores